In [1]:
import os
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.optimize import curve_fit
from joblib import dump, load


import sys
sys.path.insert(0, '../')

from utils.preprocess import *
from utils.evaluate import mapk, apk


# make MPG


In [2]:
from buyitagain.preprocess_bia import *
from buyitagain.utils_bia import *

In [3]:
folder = '../../../h-and-m-recommender/data/'

art_df = pd.read_csv(os.path.join(folder, "articles.csv"))
cus_df = pd.read_csv(os.path.join(folder, "customers.csv"))
trans_df = pd.read_csv(os.path.join(folder, "transactions_train.csv"))

trans_df = make_weeknum_col(trans_df)
# week_num 0 is latest week of the transaction
train_df, val_df = split_train_valid(trans_df, 0)
# val_df = valid2submission(val_df)

In [4]:
col = 'article_id'
rcp_threshold = 0.0
min_num_purchased = 0

In [5]:

use_cols = ['t_dat', 'customer_id', 'article_id']
train_df = train_df[use_cols]

art_df['idxgrp_idx_prdtyp'] = art_df['index_group_name'] + '_' + art_df['index_name'] + '_' + art_df['product_type_name'] 

use_cols = ['article_id', 'prod_name', 'idxgrp_idx_prdtyp']
sample_art_df = art_df[use_cols]
train_df = pd.merge(train_df, sample_art_df, how='left', on='article_id')

train_df['t_dat'] = pd.to_datetime(train_df['t_dat'])
train_df['trans_idx'] = train_df.index

col_train_df = train_df.drop_duplicates(subset=["t_dat", "customer_id", col], keep='last')

col_train_df = make_time_interval_col(col_train_df, col)

col_g_df = groupby_cid_artinfo(col_train_df, col)

col_g_df = make_rcp_df(col_g_df, col)



In [6]:
rp_col = get_repeat_purchasable(col_g_df, rcp_threshold, denom_customer_num=1)

use_cols = ['article_id', 'prod_name', 'idxgrp_idx_prdtyp']
if col == 'article_id':
    rp_col = pd.merge(rp_col, art_df[use_cols], 
                      on=col, how='left')
    rp_col.sort_values('rcp', ascending=False).head(23)

col_train_df = col_train_df[col_train_df[col].isin(rp_col[col])]


재구매가능 상품의 최소 고객수: 1


In [ ]:
# error_aids = []

# def get_shape_rate(row, artinfo_train_df, col):
#     cur_aid =  row[col]
#     # 이걸 groupby로 변경
#     cur_dist = artinfo_train_df[artinfo_train_df[col] == cur_aid][f'{col}_dist']
#     cur_dist = cur_dist[~np.isnan(cur_dist)]
#     shape, rate = np.nan, np.nan
#     try:
#         shape, _, scale = stats.gamma.fit(cur_dist) # shape, loc, scale
#         rate = (1/scale)
#     except:
#         error_aids.append(cur_aid)
#     finally:
#         return shape, rate



# from tqdm import tqdm
# tqdm.pandas()
# rp_col[['shape', 'rate']] = rp_col.progress_apply(lambda x: get_shape_rate(x, col_train_df, col), axis=1, result_type='expand')

# rp_col.to_csv('./rp_col.csv', index=False)




In [7]:
unique_idxgrp = rp_col.idxgrp_idx_prdtyp.unique()

dist_df = col_train_df[~col_train_df.article_id_dist.isna()][['idxgrp_idx_prdtyp', 'article_id_dist']]
dist_gb = pd.DataFrame(dist_df.groupby('idxgrp_idx_prdtyp')['article_id_dist'].apply(list))

def apply_gamma_fit(x):
    try :
        shape, _, scale = stats.gamma.fit(x['article_id_dist'])
        rate = (1/scale)
    except:
        shape, rate = np.nan, np.nan
    return shape, rate

dist_gb[['shape', 'rate']] = dist_gb.apply(lambda x: apply_gamma_fit(x),axis=1, result_type='expand')

/Users/nys/miniforge3/envs/torch/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:406: RuntimeWarning: invalid value encountered in double_scalars
  return m3 / np.power(m2, 1.5)


In [8]:
dist_gb['len_dist'] = dist_gb['article_id_dist'].apply(len)
dist_gb[['shape', 'rate','len_dist', 'article_id_dist']].sort_values(by= 'shape', ascending=False).head(30)
dist_gb[['shape', 'rate','len_dist', 'article_id_dist']].tail()
dist_gb = dist_gb.reset_index()

dist_gb.drop('article_id_dist', axis=1, inplace=True)

In [9]:
# rp_col = pd.read_csv('./rp_col.csv')
rp_col = pd.merge(rp_col, dist_gb, how='left', on='idxgrp_idx_prdtyp',
                 suffixes=('_aid', '_idxgrp'))

In [11]:


col = 'article_id'
rcp_threshold = 0.0
min_num_purchased = 0


rp_cid_col_df = get_rp_cid_col_df(col_train_df, col)

rp_cid_col_df = rp_cid_col_df[rp_cid_col_df.cnt >= min_num_purchased]
rp_cid_col_df = pd.merge(rp_cid_col_df, rp_col, on=col, how='left')


rp_cid_col_df = make_date_info_col(val_df, rp_cid_col_df)

rp_cid_col_df.t_min = rp_cid_col_df.t_min / np.timedelta64(1, 'D')
rp_cid_col_df.t_max = rp_cid_col_df.t_max / np.timedelta64(1, 'D')
rp_cid_col_df.t_pg_min = rp_cid_col_df.t_pg_min / np.timedelta64(1, 'D')
rp_cid_col_df.t_pg_max = rp_cid_col_df.t_pg_max / np.timedelta64(1, 'D')



/Users/nys/Desktop/h-and-m-recommender/src/trending_product_weekly/../buyitagain/preprocess_bia.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['t_dat'] = pd.to_datetime(val_df.t_dat)


- t_mean이 없는 것
    - 최대 시간 간격으로 설정

- shape, rate 감마피팅이 안된 것 -> gamma prior 가 작도록
    - shape 최소값
    - rate 최대값

    

- 감마피팅을
- 아이템 특성별로 하는 것으로 변경한다

In [12]:
# check NaN prop
print(rp_cid_col_df.t_mean.isna().mean())
print(rp_cid_col_df.shape_aid.isna().mean())
print(rp_cid_col_df.shape_idxgrp.isna().mean())

0.9569884585401099
0.028953320784134772
3.6658870952408115e-05


In [13]:
max_interval = (train_df.t_dat.max() - train_df.t_dat.min()) / np.timedelta64(1, 'D')
min_shape = rp_cid_col_df['shape_idxgrp'].min()
max_rate = rp_cid_col_df['rate_idxgrp'].max()

rp_cid_col_df['t_mean'] = rp_cid_col_df['t_mean'].fillna(max_interval)
# rp_cid_col_df['shape_idxgrp'] = rp_cid_col_df['shape_idxgrp'].fillna(min_shape)
# rp_cid_col_df['rate_idxgrp'] = rp_cid_col_df['rate_idxgrp'].fillna(max_rate)



In [14]:

rp_cid_col_df['t_min_do_pg'] = rp_cid_col_df.t_min > (2 * rp_cid_col_df.t_mean)

rp_cid_col_df.loc[rp_cid_col_df.t_min_do_pg, 'lambda_min'] = (rp_cid_col_df.cnt + rp_cid_col_df['shape_idxgrp']) \
                                                            / (rp_cid_col_df.t_pg_min + rp_cid_col_df.rate_idxgrp)

rp_cid_col_df.loc[~rp_cid_col_df.t_min_do_pg, 'lambda_min'] = (rp_cid_col_df.cnt + rp_cid_col_df['shape_idxgrp']) \
                                                             / (rp_cid_col_df.t_purch + rp_cid_col_df.rate_idxgrp\
                                                               + (2* abs(rp_cid_col_df.t_mean - rp_cid_col_df.t_min)))


rp_cid_col_df['t_max_do_pg'] = rp_cid_col_df.t_max > (2 * rp_cid_col_df.t_mean)


rp_cid_col_df.loc[rp_cid_col_df.t_max_do_pg, 'lambda_max'] = (rp_cid_col_df.cnt + rp_cid_col_df['shape_idxgrp']) \
                                                            / (rp_cid_col_df.t_pg_max + rp_cid_col_df.rate_idxgrp)

rp_cid_col_df.loc[~rp_cid_col_df.t_max_do_pg, 'lambda_max'] = (rp_cid_col_df.cnt + rp_cid_col_df['shape_idxgrp']) \
                                                             / (rp_cid_col_df.t_purch + rp_cid_col_df.rate_idxgrp\
                                                               + (2* abs(rp_cid_col_df.t_mean - rp_cid_col_df.t_max)))




In [15]:
rp_cid_col_df['R_ac_min'] = rp_cid_col_df['lambda_min'] * np.exp(rp_cid_col_df['lambda_min'])
rp_cid_col_df['R_ac_max'] = rp_cid_col_df['lambda_max'] * np.exp(rp_cid_col_df['lambda_max'])



/Users/nys/miniforge3/envs/torch/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nys/miniforge3/envs/torch/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:

# purchase product density
rp_cid_col_df['ppd_min'] = rp_cid_col_df['rcp'] * rp_cid_col_df['R_ac_min']
rp_cid_col_df['ppd_max'] = rp_cid_col_df['rcp'] * rp_cid_col_df['R_ac_max']

rp_cid_col_df = rp_cid_col_df.sort_values('ppd_max', ascending=False)


In [17]:
gc.collect()

13

In [18]:
rp_cid_col_df.to_csv('./rp_cid_col_df.csv', index=False)


In [20]:
tmp = pd.read_csv('./tmp.csv')

In [21]:
for col in rp_cid_col_df.columns:
    print(col, end= '  ')
print()
print('########'*10)
for col in tmp.columns:
    print(col, end= '  ')

customer_id  article_id  t_mean  t_purch  t_last  t_first  cnt  denom_customer  num_customer  rcp  prod_name  idxgrp_idx_prdtyp  shape_aid  rate_aid  shape_idxgrp  rate_idxgrp  len_dist  cur_date_min  cur_date_max  t_min  t_max  t_pg_min  t_pg_max  t_min_do_pg  lambda_min  t_max_do_pg  lambda_max  R_ac_min  R_ac_max  ppd_min  ppd_max  
################################################################################
t_dat  customer_id  article_id  price  sales_channel_id  week_num  ldbw  count  count_targ  quotient  x  dummy_1  y  dummy_0  value  

# Apply MPG
- Apply MPG instead of line fitting

- y 값은 전체 거래에 대하여 재구매가 일어날 수 있는 값
    - 이것을 상품 & 고객 별로 다르게 MPG모델의 확률값으로 변경

In [1]:
import os
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.optimize import curve_fit
from joblib import dump, load


import sys
sys.path.insert(0, '../')

from utils.preprocess import *
from utils.evaluate import mapk, apk


In [2]:
rp_cols = ['customer_id' , 'article_id', 'rcp', 
           'R_ac_min', 'R_ac_max', 'ppd_min' , 'ppd_max']

tmp_cols = ['t_dat','customer_id' , 'article_id'
            ,'quotient' , 'value', 'y']

rp_cid_col_df = pd.read_csv('./rp_cid_col_df.csv', usecols=rp_cols)

tmp = pd.read_csv('./tmp.csv', usecols=tmp_cols)

# folder = '../../../h-and-m-recommender/data/'
# art_df = pd.read_csv(os.path.join(folder, "articles.csv"))
# cus_df = pd.read_csv(os.path.join(folder, "customers.csv"))
# trans_df = pd.read_csv(os.path.join(folder, "transactions_train.csv"))
# trans_df = make_weeknum_col(trans_df)
# week_num 0 is latest week of the transaction
# _, val_df = split_train_valid(trans_df, 0)
# val_df['t_dat'] = pd.to_datetime(val_df.t_dat)
# validation_df = valid2submission(val_df)
# validation_df.to_csv('./validation_df.csv', index=False)

validation_df = pd.read_csv('./validation_df.csv')



In [3]:

rp_cid_col_df['ppd'] = (rp_cid_col_df['ppd_min'] + rp_cid_col_df['ppd_max'])/2

new_tmp = pd.merge(tmp, rp_cid_col_df, how='left', on=['customer_id', 'article_id'])


In [4]:

new_tmp['ppd_cutmax'] = new_tmp['ppd']
new_tmp['ppd_cutmean'] = new_tmp['ppd']

print(new_tmp.ppd.isna().sum())

new_tmp.loc[new_tmp['ppd_cutmax'] >= 1.0, 'ppd_cutmax'] = 1.0
new_tmp.loc[new_tmp['ppd'] >= 1.0, 'ppd_cutmean'] = np.nan
print(new_tmp.ppd_cutmean.isna().sum())
ppd_nanmean = np.nanmean(new_tmp.ppd_cutmean)
new_tmp.loc[new_tmp['ppd'] >= 1.0, 'ppd_cutmean'] = ppd_nanmean
print(new_tmp.ppd_cutmean.isna().sum())


127450
136362
127450


In [5]:
for col in ['ppd_cutmax','ppd_cutmean']:
    new_tmp[f'{col}_mean'] = new_tmp[col].fillna(new_tmp[col].mean())
    new_tmp[f'{col}_min'] = new_tmp[col].fillna(new_tmp[col].min())
    new_tmp[f'{col}_zero'] = new_tmp[col].fillna(0)    
    for fill_val in ['mean', 'min', 'zero']:
        new_tmp[f'val_{col}_{fill_val}'] = new_tmp['value'] * new_tmp[f'{col}_{fill_val}']
        new_tmp[f'quot_{col}_{fill_val}'] = new_tmp['quotient'] * new_tmp[f'{col}_{fill_val}']


In [8]:
import warnings
warnings.filterwarnings("ignore")

results = []
for multiply_by in ['val', 'quot']:
    for col in ['ppd_cutmax','ppd_cutmean']:
        for fill_val in ['mean', 'min', 'zero']:
            for cut_off in [0.0, 0.1, 0.25]:

                new_value_col = f'{multiply_by}_{col}_{fill_val}'
                cut_value = new_tmp[new_value_col].quantile(cut_off) 
                cut_new_tmp = new_tmp.loc[new_tmp[new_value_col] >= cut_value]

                cut_new_tmp['rank'] = cut_new_tmp.groupby("customer_id")[new_value_col]\
                                                 .rank("dense", ascending=False)
                cut_new_tmp = cut_new_tmp.loc[cut_new_tmp['rank'] <= 12]

                purchase_df = cut_new_tmp.sort_values(['customer_id', new_value_col], ascending = False).reset_index(drop = True)

                purchase_df['prediction'] = '0' + purchase_df['article_id'].astype(str) + ' '
                purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
                purchase_df['prediction'] = purchase_df['prediction'].str.strip()

                merged = pd.merge(purchase_df, validation_df, on='customer_id', how='right')
                merged['prediction'].fillna('', inplace=True)

                metric = mapk(
                    merged['article_id'].map(lambda x: x.split()), 
                    merged['prediction'].map(lambda x: x.split()), 
                    k=12
                )
                results.append([multiply_by, col, fill_val, cut_off, metric])
                print(multiply_by, col, fill_val, cut_off, metric)
                del merged, purchase_df, cut_new_tmp; gc.collect()



val ppd_cutmax mean 0.0 0.019248947399243398
val ppd_cutmax mean 0.1 0.019244460506492554
val ppd_cutmax mean 0.25 0.01921311178613624
val ppd_cutmax min 0.0 0.019141156324509436
val ppd_cutmax min 0.1 0.019137172769086417
val ppd_cutmax min 0.25 0.019095087863527736
val ppd_cutmax zero 0.0 0.019069382404406556
val ppd_cutmax zero 0.1 0.01895453971333242
val ppd_cutmax zero 0.25 0.018922949391058753
val ppd_cutmean mean 0.0 0.019406577900092165
val ppd_cutmean mean 0.1 0.019402091007341332
val ppd_cutmean mean 0.25 0.019370742286985012
val ppd_cutmean min 0.0 0.019212870263943915
val ppd_cutmean min 0.1 0.019208886708520893
val ppd_cutmean min 0.25 0.019166801802962215
val ppd_cutmean zero 0.0 0.019141096343841035
val ppd_cutmean zero 0.1 0.019026253652766893
val ppd_cutmean zero 0.25 0.01899466333049322
quot ppd_cutmax mean 0.0 0.013795579119357615
quot ppd_cutmax mean 0.1 0.013791092226606776
quot ppd_cutmax mean 0.25 0.013712950914991036
quot ppd_cutmax min 0.0 0.01324267956662207
q

In [12]:
results_df = pd.DataFrame(results)
results_df.columns = ['multiply_by','col', 'fill_val', 'cut_off', 'map@12']

In [13]:
results_df.sort_values('map@12',ascending=False)

,multiply_by,col,fill_val,cut_off,map@12
9,val,ppd_cutmean,mean,0.00,0.019407
10,val,ppd_cutmean,mean,0.10,0.019402
11,val,ppd_cutmean,mean,0.25,0.019371
0,val,ppd_cutmax,mean,0.00,0.019249
1,val,ppd_cutmax,mean,0.10,0.019244
2,val,ppd_cutmax,mean,0.25,0.019213
12,val,ppd_cutmean,min,0.00,0.019213
13,val,ppd_cutmean,min,0.10,0.019209
14,val,ppd_cutmean,min,0.25,0.019167
3,val,ppd_cutmax,min,0.00,0.019141
